In [2]:
import numpy as np
import itertools as it
from PGM_PyLib.MRF import RMRFwO as mrf
import requests

In [3]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

# Assuming relational_probabilities is global
def Uf(rmrf, observation, row, col):
    """
    Local energy function for a bounding box in a complete graph (3 rows, 1 column RMRF).
    
    This function calculates the energy based on the current state of the bounding box
    and its relationships with all other bounding boxes (complete graph assumption).
    Additionally, it considers n-tuples (n >= 3) by averaging the weights for all such tuples.
    Observation probabilities are also factored into the energy calculation.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (3 rows, 1 column, states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index (0-based, corresponding to the bounding box)
    - col: int, column index (should always be 0 since it's a 1-column matrix)

    Returns:
    - float, the calculated energy for the bounding box at row `row`
    """
    current_state = rmrf[row, col]  # Get the current state of the bounding box
    energy = 0.0
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_prob = observation[row][current_state]  # Probability for the current state
    energy -= np.log(obs_prob)  # Add the energy contribution from the observation
    
    # 2. Calculate the pairwise (2-tuple) energy from the relationships with other bounding boxes (complete graph)
    num_boxes = rmrf.shape[0]  # Number of bounding boxes (rows in rmrf)

    for other_row in range(num_boxes):
        if other_row != row:  # Avoid self-relations
            other_state = rmrf[other_row, col]  # Get the state of the other bounding box
            # Get the potential (weight) from the relational probability matrix
            weight = relational_probabilities[current_state, other_state]
            energy -= np.log(weight)  # Add the energy contribution from the relationship

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples (where n >= 3)
    for n in range(3, num_boxes + 1):  # Start from n=3, up to n=num_boxes
        # Get all combinations of bounding boxes of size n
        tuples = list(it.combinations(range(num_boxes), n))
        
        # For each n-tuple, calculate the average relational probability
        for t in tuples:
            # Extract the relational probabilities for all pairs in the tuple
            pairwise_weights = [
                relational_probabilities[rmrf[i, col], rmrf[j, col]]
                for i, j in it.combinations(t, 2)
            ]
            
            # Calculate the mean of the pairwise weights using NumPy
            if pairwise_weights:  # Ensure there are weights to average
                average_weight = np.mean(pairwise_weights)
                energy -= np.log(average_weight)  # Subtract the energy from the average weight
    
    return energy

In [4]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_probabilities = generate_symmetric_matrix(4)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
bounding_box_probabilities = generate_random_probabilities(num_boxes=3, num_states=4)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference (Bounding Box States):")
print(result)  # This will be a 3x1 matrix with the final state for each bounding box

Initial RMRF
 [[0]
 [0]
 [0]]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.74648619 0.1736355  0.87353087]
 [0.74648619 1.         0.51168884 0.75138824]
 [0.1736355  0.51168884 1.         0.3219275 ]
 [0.87353087 0.75138824 0.3219275  1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.41734541 0.47814125 0.07133188 0.03318146]
 [0.43042542 0.05435965 0.21724475 0.29797018]
 [0.23438132 0.28564797 0.09575869 0.38421203]] 

Succesfully finish, iteration: 1

Final RMRF after inference (Bounding Box States):
[[0]
 [0]
 [3]]


In [2]:
import requests

# Define the list of concepts
concepts = ['tree', 'glass', 'cup', 'car']

# API base URL and relation type
base_url = "http://api.conceptnet.io/query"
relation = "/r/AtLocation"

# Initialize an empty 4x4 matrix to store the weights
weights_matrix = [[None for _ in range(4)] for _ in range(4)]

# Function to get the weight of AtLocation relation between two concepts
def get_weight(concept1, concept2):
    start = f"/c/en/{concept1}"
    end = f"/c/en/{concept2}"
    # Query the API for the AtLocation relationship
    query_url = f"{base_url}?start={start}&end={end}&rel={relation}"
    response = requests.get(query_url).json()
    
    # Extract the weight from the first edge if it exists
    if response['edges']:
        return response['edges'][0]['weight']
    return None

# Fill the matrix with weights
for i, concept1 in enumerate(concepts):
    for j, concept2 in enumerate(concepts):
        if i != j:
            weight = get_weight(concept1, concept2)
            weights_matrix[i][j] = weight

# Print the resulting matrix
for row in weights_matrix:
    print(row)

[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]


In [16]:
# Function to check if an AtLocation relation exists between two concepts
def check_atlocation_relationship(concept1, concept2):
    base_url = "http://api.conceptnet.io/query"
    
    # Construct the URIs for the two concepts
    start = f"/c/en/{concept1}"
    end = f"/c/en/{concept2}"
    
    # Query the API for relations between these two concepts
    query_url = f"{base_url}?start={start}&end={end}"
    response = requests.get(query_url).json()
    
    # Debug: print the response to check what's being returned
    print(f"API Response for {concept1} and {concept2}:")
    print(response)
    
    # Check if there is any edge with the "AtLocation" relation
    for edge in response.get('edges', []):
        if edge['rel']['@id'] == '/r/AtLocation':
            return True, edge['weight']  # Return True and the weight if AtLocation exists
    
    # If no AtLocation relation is found
    return False, None

# Test for the pair "tree" and "glass"
concept1 = "water"
concept2 = "glass"

exists, weight = check_atlocation_relationship(concept1, concept2)

if exists:
    print(f"The 'AtLocation' relation exists between {concept1} and {concept2} with a weight of {weight}.")
else:
    print(f"No direct 'AtLocation' relation exists between {concept1} and {concept2}.")

API Response for water and glass:
{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'], '@id': '/query?start=/c/en/water&end=/c/en/glass', 'edges': [{'@id': '/a/[/r/AtLocation/,/c/en/water/,/c/en/glass/]', '@type': 'Edge', 'dataset': '/d/conceptnet/4/en', 'end': {'@id': '/c/en/glass', '@type': 'Node', 'label': 'a glass', 'language': 'en', 'term': '/c/en/glass'}, 'license': 'cc:by/4.0', 'rel': {'@id': '/r/AtLocation', '@type': 'Relation', 'label': 'AtLocation'}, 'sources': [{'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/narcisse/]', '@type': 'Source', 'activity': '/s/activity/omcs/omcs1_possibly_free_text', 'contributor': '/s/contributor/omcs/narcisse'}, {'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/ovan4/]', '@type': 'Source', 'activity': '/s/activity/omcs/omcs1_possibly_free_text', 'contributor': '/s/contributor/omcs/ovan4'}, {'@id': '/and/[/s/activity/omcs/vote/,/s/contributor/omcs/annedog/]', '@type

In [22]:
# Define the endpoints for the two concepts
glass_url = 'http://api.conceptnet.io/c/en/glass'
table_url = 'http://api.conceptnet.io/c/en/table'

# Fetch the data for glass and table
glass_data = requests.get(glass_url).json()
table_data = requests.get(table_url).json()

# Function to find the relationship between two concepts
def find_relationship(concept1_data, concept2):
    for edge in concept1_data['edges']:
        if edge['end']['label'] == concept2 or edge['start']['label'] == concept2:
            rel_type = edge['rel']['label']
            weight = edge['weight']
            return f"Relationship: {rel_type}, Weight: {weight}"
    return f"No direct relationship found between {concept2} and the given concept."

# Check relationships between glass and table
relationship = find_relationship(glass_data, "water")

print(relationship)

Relationship: AtLocation, Weight: 4.0


In [18]:
glass_data['edges']

[{'@id': '/a/[/r/RelatedTo/,/c/en/window/,/c/en/glass/]',
  '@type': 'Edge',
  'dataset': '/d/verbosity',
  'end': {'@id': '/c/en/glass',
   '@type': 'Node',
   'label': 'glass',
   'language': 'en',
   'term': '/c/en/glass'},
  'license': 'cc:by/4.0',
  'rel': {'@id': '/r/RelatedTo', '@type': 'Relation', 'label': 'RelatedTo'},
  'sources': [{'@id': '/and/[/s/process/split_words/,/s/resource/verbosity/]',
    '@type': 'Source',
    'contributor': '/s/resource/verbosity',
    'process': '/s/process/split_words'},
   {'@id': '/s/resource/verbosity',
    '@type': 'Source',
    'contributor': '/s/resource/verbosity'}],
  'start': {'@id': '/c/en/window',
   '@type': 'Node',
   'label': 'window',
   'language': 'en',
   'term': '/c/en/window'},
  'surfaceText': '[[window]] is related to [[glass]]',
  'weight': 8.952764936040708},
 {'@id': '/a/[/r/RelatedTo/,/c/en/glass/,/c/en/window/]',
  '@type': 'Edge',
  'dataset': '/d/verbosity',
  'end': {'@id': '/c/en/window',
   '@type': 'Node',
   'l

In [21]:
#Show all the weight and concepts for AtLocation of glass_data['edges']
for edge in glass_data['edges']:
    if edge['rel']['label'] == 'AtLocation':
        print(f"Weight: {edge['weight']}, Concept: {edge['end']['label']}")


Weight: 4.0, Concept: the cabinet
Weight: 4.0, Concept: a glass


In [31]:
# Define the initial endpoint for the concept "glass"
glass_url = 'http://api.conceptnet.io/c/en/glass'

# Function to collect edges with 'AtLocation' relation, limiting to a few pages
def get_atlocation_edges(concept_url, max_pages=10):
    atlocation_edges = []
    next_page_url = concept_url  # Start with the initial concept URL
    page_count = 0  # Track the number of pages retrieved

    while next_page_url and page_count < max_pages:
        # Fetch data from the current URL
        response = requests.get(next_page_url).json()

        # Filter edges where the relation is 'AtLocation'
        for edge in response['edges']:
            if edge['rel']['label'] == 'AtLocation':
                atlocation_edges.append(edge)
        
        # Check if there's a next page
        if 'view' in response and 'nextPage' in response['view']:
            next_page_url = 'http://api.conceptnet.io' + response['view']['nextPage']
            page_count += 1  # Increment the page counter
        else:
            next_page_url = None  # No more pages to fetch

    return atlocation_edges

# Fetch 'AtLocation' edges for the concept "glass" with a limit of 3 pages
atlocation_edges = get_atlocation_edges(glass_url, max_pages=100)

# Print the number of 'AtLocation' edges retrieved
print(f"Total 'AtLocation' edges retrieved for 'glass': {len(atlocation_edges)}")

#Show all the weight and concepts for AtLocation of glass
for edge in atlocation_edges:
    print(f"Weight: {edge['weight']}, Concept: {edge['end']['label']}")

Total 'AtLocation' edges retrieved for 'glass': 19
Weight: 4.0, Concept: the cabinet
Weight: 4.0, Concept: a glass
Weight: 3.4641016151377544, Concept: a window
Weight: 2.82842712474619, Concept: a kitchen
Weight: 2.0, Concept: the table
Weight: 1.0, Concept: a dining room
Weight: 1.0, Concept: a street
Weight: 1.0, Concept: a
Weight: 1.0, Concept: a
Weight: 1.0, Concept: a
Weight: 1.0, Concept: a glass
Weight: 1.0, Concept: a glass
Weight: 1.0, Concept: a sink
Weight: 1.0, Concept: a glass
Weight: 1.0, Concept: a cuboard
Weight: 1.0, Concept: your kitchen cupboard
Weight: 1.0, Concept: a glass
Weight: 1.0, Concept: a glass
Weight: 1.0, Concept: a water cooler


In [5]:
# Define the search endpoint for fetching edges with the 'AtLocation' relation
search_url = 'http://api.conceptnet.io/query'

# Function to search for edges with 'AtLocation' relation
def get_atlocation_edges_via_search(concept, max_limit=10000000):
    params = {
        'start': f'/c/en/{concept}',  # Start node is the concept (glass)
        'rel': '/r/AtLocation',       # Only fetch edges with the 'AtLocation' relation
        'limit': max_limit            # Limit the number of edges for performance
    }
    # Make a request to the ConceptNet search endpoint
    response = requests.get(search_url, params=params).json()

    return response['edges']

# Fetch 'AtLocation' edges for the concept "glass" with a limit
atlocation_edges = get_atlocation_edges_via_search('glass', max_limit=10000000)

# Print the number of 'AtLocation' edges retrieved
print(f"Total 'AtLocation' edges retrieved for 'glass': {len(atlocation_edges)}")

#Show all the weight and concepts for AtLocation of glass
for edge in atlocation_edges:
    print(f"Weight: {edge['weight']}, Concept: {edge['end']['label']}")

Total 'AtLocation' edges retrieved for 'glass': 13
Weight: 4.0, Concept: the cabinet
Weight: 3.4641016151377544, Concept: a window
Weight: 2.82842712474619, Concept: a kitchen
Weight: 2.0, Concept: the table
Weight: 1.0, Concept: a water cooler
Weight: 1.0, Concept: a dishwasher
Weight: 1.0, Concept: a hand
Weight: 1.0, Concept: a sink
Weight: 1.0, Concept: a cuboard
Weight: 1.0, Concept: your kitchen cupboard
Weight: 1.0, Concept: a
Weight: 1.0, Concept: a dining room
Weight: 1.0, Concept: a street


In [8]:
# Define the search endpoint for fetching edges with the 'AtLocation' relation
search_url = 'http://api.conceptnet.io/query'

# Function to search for edges with 'AtLocation' relation, including 'glass' as both start and end
def get_atlocation_edges_both_directions(concept, max_limit=50):
    params_start = {
        'start': f'/c/en/{concept}',  # Fetch edges where 'glass' is the start
        'rel': '/r/AtLocation',
        'limit': max_limit
    }
    params_end = {
        'end': f'/c/en/{concept}',    # Fetch edges where 'glass' is the end
        'rel': '/r/AtLocation',
        'limit': max_limit
    }

    # Fetch edges where 'glass' is the start
    response_start = requests.get(search_url, params=params_start).json()
    edges_start = response_start['edges']

    # Fetch edges where 'glass' is the end
    response_end = requests.get(search_url, params=params_end).json()
    edges_end = response_end['edges']

    # Combine both sets of edges
    return edges_start + edges_end

# Fetch 'AtLocation' edges for the concept "glass" with a limit
atlocation_edges = get_atlocation_edges_both_directions('glass', max_limit=100000)

# Print the number of 'AtLocation' edges retrieved
print(f"Total 'AtLocation' edges retrieved for 'glass': {len(atlocation_edges)}")

#Show all the weight and concepts for AtLocation of glas and show diretcion in a visual way
for edge in atlocation_edges:
    start = edge['start']['label'].split('/')[-1]
    end = edge['end']['label'].split('/')[-1]
    weight = edge['weight']
    print(f"{start} -> {end} (Weight: {weight})")

Total 'AtLocation' edges retrieved for 'glass': 17
a glass -> the cabinet (Weight: 4.0)
a glass -> a window (Weight: 3.4641016151377544)
a glass -> a kitchen (Weight: 2.82842712474619)
a glass -> the table (Weight: 2.0)
a glass -> a water cooler (Weight: 1.0)
a glass -> a dishwasher (Weight: 1.0)
a glass -> a hand (Weight: 1.0)
a glass -> a sink (Weight: 1.0)
a glass -> a cuboard (Weight: 1.0)
a glass -> your kitchen cupboard (Weight: 1.0)
a glass -> a (Weight: 1.0)
a glass -> a dining room (Weight: 1.0)
glass -> a street (Weight: 1.0)
water -> a glass (Weight: 4.0)
an overflow -> a glass (Weight: 1.0)
ice -> a glass (Weight: 1.0)
ice cubes -> a glass (Weight: 1.0)
